# 대한민국 도시 고유 ID 주기
## Json파일을 이용해서 Folium으로 그리든, Cartogram으로 표현하든 두 방법 다 각 지역을 대표하는 고유 ID를 만들어줘야 한다.

In [83]:
import pandas as pd
import numpy as np

In [84]:
pop = pd.read_csv('./result/성별, 연령별 인구 - 시도(2019).csv')
pop.head()

,광역시도,시도,20~39세계,20~39세남자,20~39세여자,65세 이상계,65세 이상남자,65세 이상여자,인구수계,인구수남자,인구수여자,소멸 비율,소멸 위기 지역
0,강원,강릉시,46008,25015,20993,40497,17378,23119,203057,101154,101903,0.518384,False
1,강원,고성군,4006,2307,1699,7005,2957,4048,23621,11933,11688,0.242541,True
2,강원,동해시,16311,8801,7510,16152,7075,9077,83888,42093,41795,0.464958,True
3,강원,삼척시,12428,7148,5280,14953,6314,8639,61833,31630,30203,0.353106,True
4,강원,속초시,15928,8517,7411,14010,5866,8144,76302,37488,38814,0.528979,False


## 각 지역 고유 ID를 만들기 전에 지역 이름이 겹치는 게 있는지 먼저 확인

In [85]:
pop['시도'].unique()

array([' 강릉시', ' 고성군', ' 동해시', ' 삼척시', ' 속초시', ' 양구군', ' 양양군', ' 영월군',
       ' 원주시', ' 인제군', ' 정선군', ' 철원군', ' 춘천시', ' 태백시', ' 평창군', ' 홍천군',
       ' 화천군', ' 횡성군', ' 가평군', ' 고양시', ' 과천시', ' 광명시', ' 광주시', ' 구리시',
       ' 군포시', ' 김포시', ' 남양주시', ' 동두천시', ' 부천시', ' 성남시', ' 수원시', ' 시흥시',
       ' 안산시', ' 안성시', ' 안양시', ' 양주시', ' 양평군', ' 여주시', ' 연천군', ' 오산시',
       ' 용인시', ' 의왕시', ' 의정부시', ' 이천시', ' 파주시', ' 평택시', ' 포천시', ' 하남시',
       ' 화성시', ' 거제시', ' 거창군', ' 김해시', ' 남해군', ' 밀양시', ' 사천시', ' 산청군',
       ' 양산시', ' 의령군', ' 진주시', ' 창녕군', ' 창원시', ' 통영시', ' 하동군', ' 함안군',
       ' 함양군', ' 합천군', ' 경산시', ' 경주시', ' 고령군', ' 구미시', ' 군위군', ' 김천시',
       ' 문경시', ' 봉화군', ' 상주시', ' 성주군', ' 안동시', ' 영덕군', ' 영양군', ' 영주시',
       ' 영천시', ' 예천군', ' 울릉군', ' 울진군', ' 의성군', ' 청도군', ' 청송군', ' 칠곡군',
       ' 포항시', ' 광산구', ' 남구', ' 동구', ' 북구', ' 서구', ' 달서구', ' 달성군', ' 수성구',
       ' 중구', ' 대덕구', ' 유성구', ' 강서구', ' 금정구', ' 기장군', ' 동래구', ' 부산진구',
       ' 사상구', ' 사하구', ' 수영구', ' 연제구', ' 영도구', ' 해운대구', ' 강남구', ' 강동구'

## '광역시도' 값과 '시도' 값을 합치면 고유 ID (서울 강남, 인천 부평)을 만들 수 있다.
- 인천, 부산 등 광역시의 구(자치구) 도 있으나, 안양시나 수원시에 있는 구(행정구) 도 있다.
- 광역시도의 자치구는 '서울 중구' 처럼 나오긴 하지만, 행정구의 경우는 '안양시 동안구'가 되지 않고 '경기도 동안구'가 됨. 이걸 어떻게 해결할 수 있을까?

## 먼저 광역시가 아니면서 구를 가지고 있는 시와 그 행정구를 Dictionary로 만듦

In [86]:
si_name = [None] * len(pop) # 264
si_name[:5]

[None, None, None, None, None]

In [87]:
tmp_gu_dict = {'수원':['장안구', '권선구', '팔달구', '영통구'], 
               '성남':['수정구', '중원구', '분당구'], 
               '안양':['만안구', '동안구'], 
               '안산':['상록구', '단원구'], 
               '고양':['덕양구', '일산동구', '일산서구'], 
               '용인':['처인구', '기흥구', '수지구'], 
               '청주':['상당구', '서원구', '흥덕구', '청원구'], 
               '천안':['동남구', '서북구'], 
               '전주':['완산구', '덕진구'], 
               '포항':['남구', '북구'], 
               '창원':['의창구', '성산구', '진해구', '마산합포구', '마산회원구']}

In [88]:
pop['광역시도'].unique()

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

1. 광역시도에 있는 이름의 끝 세자가 '광역시','특별시','자치시'로 끝나지 않으면 일반 시 혹은 군으로 처리
2. 강원도 고성군과 경상남도 고성군 처리
3. 세종특별자치시는 세종으로
4. 광역시도에서 앞 두 글자(서울특별시 => 서울), 나머지는 광역시도에서 앞 두 글자(서울특별시)와 시도에서 두 글자인 경우 모두, 아니면 앞 두 글자만 선택하며 고유 ID를 만듦

In [89]:
pop.head()

,광역시도,시도,20~39세계,20~39세남자,20~39세여자,65세 이상계,65세 이상남자,65세 이상여자,인구수계,인구수남자,인구수여자,소멸 비율,소멸 위기 지역
0,강원,강릉시,46008,25015,20993,40497,17378,23119,203057,101154,101903,0.518384,False
1,강원,고성군,4006,2307,1699,7005,2957,4048,23621,11933,11688,0.242541,True
2,강원,동해시,16311,8801,7510,16152,7075,9077,83888,42093,41795,0.464958,True
3,강원,삼척시,12428,7148,5280,14953,6314,8639,61833,31630,30203,0.353106,True
4,강원,속초시,15928,8517,7411,14010,5866,8144,76302,37488,38814,0.528979,False


In [96]:
for i in pop.index:
    for keys, values in tmp_gu_dict.items():
        if pop['시도'][i] in values:
            if len(pop['시도'][i]) == 2:
                si_name[i] = keys + ' ' + pop['시도'][i]
            elif pop['시도'][i] in ['마산합포구', '마산회원구']:
                si_name[i] = keys + ' ' + pop['시도'][i][2:-1]
            else:
                si_name[i] = keys + ' ' + pop['시도'][i][:-1]
    

    if pop['광역시도'][i] not in ['서울','부산','대구','인천','광주','대전','울산']:
        if pop['시도'][i][1:-1] == '고성' and pop['광역시도'][i] == '강원':
            si_name[i] = '고성(강원)'
        elif pop['시도'][i][:-1] == '고성' and pop['광역시도'][i] == '경남':
                si_name[i] = '고성(경남)'
        else:
            si_name[i] = pop['시도'][i][1:-1]        


    else:
        if len(pop['시도'][i]) == 3:
            si_name[i] = pop['광역시도'][i] + ' ' + pop['시도'][i][1:]
        else:
            si_name[i] = pop['광역시도'][i] + ' ' + pop['시도'][i][1:-1]


In [97]:
si_name[:10]

['강릉', '고성(강원)', '동해', '삼척', '속초', '양구', '양양', '영월', '원주', '인제']

In [98]:
pop['ID'] = si_name

In [93]:
del pop['20~39세남자']
del pop['65세 이상남자']
del pop['65세 이상여자']

In [99]:
pop.head()

,광역시도,시도,20~39세계,20~39세여자,65세 이상계,인구수계,인구수남자,인구수여자,소멸 비율,소멸 위기 지역,ID
0,강원,강릉시,46008,20993,40497,203057,101154,101903,0.518384,False,강릉
1,강원,고성군,4006,1699,7005,23621,11933,11688,0.242541,True,고성(강원)
2,강원,동해시,16311,7510,16152,83888,42093,41795,0.464958,True,동해
3,강원,삼척시,12428,5280,14953,61833,31630,30203,0.353106,True,삼척
4,강원,속초시,15928,7411,14010,76302,37488,38814,0.528979,False,속초


In [100]:
pop.to_csv('./result/소멸 위기 지역.csv', sep=',', encoding = 'utf-8')